<a href="https://colab.research.google.com/github/Dhaneshkp/GraphML/blob/main/Fraud_xgb%20-%20Copy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

!pip install --upgrade certifi

!pip install --trusted-host files.pythonhosted.org optuna



In [ ]:
#os.listdir("/kaggle/input/fraud-detection")

In [ ]:
data=pd.read_csv("/kaggle/input/fraud-detection/fraudTrain.csv")

In [ ]:

def are_cc_numbers_unique(df, cc_column):

    try:
      # Check if the specified column exists in the DataFrame.
      if cc_column not in df.columns:
          print(f"Error: Column '{cc_column}' not found in the DataFrame.")
          return False

      return df[cc_column].nunique() == len(df)

    except Exception as e:
      print(f"An error occurred: {e}")
      return False


are_unique = are_cc_numbers_unique(data, 'cc_num')

if are_unique:
     print("All credit card numbers are unique.")
else:
     print("Some credit card numbers are duplicated.")

In [ ]:

import datetime

data['trans_date_trans_time'] = pd.to_datetime(data['trans_date_trans_time'])

# Extract the time of day as an integer representing minutes since midnight
data['time_of_day'] = (data['trans_date_trans_time'].dt.hour * 60) + data['trans_date_trans_time'].dt.minute


data['time_of_day_category'] = pd.cut(data['time_of_day'], bins=[0, 6*60, 12*60, 18*60, 24*60],
                                      labels=['Night', 'Morning', 'Afternoon', 'Evening'], right=False)


data.head()

In [ ]:

data['unix_time_c'] = pd.to_datetime(data['unix_time'], unit='s')

# Extract the time of day as an integer representing minutes since midnight
data['time_of_day_uc'] = (data['unix_time_c'].dt.hour * 60) + data['unix_time_c'].dt.minute

data['time_of_day_category_uc'] = pd.cut(data['time_of_day_uc'], bins=[0, 6*60, 12*60, 18*60, 24*60],
                                      labels=['Night', 'Morning', 'Afternoon', 'Evening'], right=False)

data.head()

In [ ]:
data.columns

In [ ]:
data.describe()

In [ ]:
data.describe(include='object')

In [ ]:
train_df = data.drop(['Unnamed: 0', 'trans_date_trans_time','trans_num', 'unix_time'],axis=1)


In [ ]:
train_df=train_df.sample(frac=0.5,random_state=1)# if the data us indded by a person or a cc num then this sampling isnt appropriate

In [ ]:
train_df['distance']= (train_df['lat'] - train_df['merch_lat'])**2 + (train_df['long'] - train_df['merch_long'])**2

In [ ]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = ' '.join([word for word in text.split() if word not in stopwords.words('english') and not word.isdigit()])
    return text

# Apply cleaning to relevant columns
#train_df['merchant'] = train_df['merchant'].apply(clean_text)
#train_df['category'] = train_df['category'].apply(clean_text)
train_df['city_state'] = train_df['city'] + '_' + train_df['state']

In [ ]:
train_df['city_state'] = train_df['city'] + '_' + train_df['state']

In [ ]:
from datetime import datetime

# Function to calculate age
def calculate_age(dob):
    dob = str(dob)
    today = datetime.today()
    dob = datetime.strptime(dob, '%Y-%m-%d')
    age = today.year - dob.year - ((today.month, today.day) < (dob.month, dob.day))
    return age

# Apply the function to the 'dob' column
train_df['age'] = data['dob'].apply(calculate_age)


In [ ]:
#train_df = train_df.dropna(subset=['dob'])

In [ ]:
categorical_columns = ['category', 'gender', 'job','city','state','merchant','street','city_state']
for column in categorical_columns:
    train_df[column] = train_df[column].astype('category')

In [ ]:
train_df.dtypes

In [ ]:
train_df.drop(['merch_lat', 'merch_long','lat', 'long'],axis=1,inplace=True)

In [ ]:
train_df.columns

In [ ]:
train_df.drop(['street','zip','city_pop'],axis=1,inplace=True)

In [ ]:
#train_df.drop(['dob'],axis=1,inplace=True)

In [ ]:
#train_df.to_csv("train_df_new.csv")

In [ ]:
#train_df=pd.read_csv("train_df_new.csv")
#train_df.drop('Unnamed: 0',axis=1,inplace=True)

In [ ]:
!pip install pykeen

In [ ]:
import networkx as nx

G = nx.Graph()

# Add nodes and edges based on the data
for index, row in train_df.iterrows():
    trans_id = f'trans_{index}'
    G.add_node(trans_id, label='transaction')
    G.add_node(row['merchant'], label='merchant')
    G.add_node(row['category'], label='category')
    G.add_node(row['city'], label='city')
    G.add_node(row['state'], label='state')
    G.add_edge(trans_id, row['merchant'], relationship='made_at')
    G.add_edge(trans_id, row['category'], relationship='belongs_to')
    G.add_edge(trans_id, row['city'], relationship='located_in')
    G.add_edge(trans_id, row['state'], relationship='in')
    G.add_node(row['time_of_day_category'] , label='time')
    G.add_edge(trans_id, row['time_of_day_category'] , relationship='time')
    #G.add_node(row['cc_num'], label='cc_num')
    #G.add_edge(trans_id, row['cc_num'], relationship='card')
    G.add_node(row['job'], label='job')
    G.add_edge(trans_id, row['job'], relationship='job')

In [ ]:
from pykeen.pipeline import pipeline
from pykeen.triples import TriplesFactory
from sklearn.model_selection import train_test_split
import torch
# Create triples for KG (subject, predicate, object)
triples = []
for index, row in train_df.iterrows():
    trans_id = f'trans_{index}'
    triples.append((trans_id, 'made_at', row['merchant']))
    triples.append((trans_id, 'belongs_to', row['category']))
    triples.append((trans_id, 'located_in', row['city']))
    triples.append((trans_id, 'in', row['state']))
    triples.append((trans_id, 'in', row['time_of_day_category']))
    triples.append((trans_id, 'job', row['job']))
    #triples.append((trans_id, 'card', row['cc_num']))
triples = np.array(triples)
# Create a PyKEEN dataset
for i in range(25):
    print(triples[i])

# Check data types of the triples
print(type(triples[0][0]), type(triples[0][1]), type(triples[0][2]))

# Remove malformed entries
valid_triples = [triple for triple in triples if len(triple) == 3]
triples = np.array(valid_triples)

# Split the triples into training and testing sets
train_triples, test_triples = train_test_split(triples, test_size=0.2, random_state=42)
train_triples=triples
# Create TriplesFactory objects for training and testing sets
train_tf = TriplesFactory.from_labeled_triples(train_triples)
test_tf = TriplesFactory.from_labeled_triples(test_triples)

# Print TriplesFactory details for debugging
print(train_tf)
print(test_tf)

In [ ]:


# Train a KG embedding model with both training and testing triples factories
result = pipeline(
    training=train_tf,
    testing=test_tf,
    model='TransE',
    training_kwargs=dict(num_epochs=10),
)
all_entities = train_tf.entity_to_id
embeddings = result.model.entity_representations[0](indices=torch.arange(train_tf.num_entities, device=result.model.device)).cpu().detach().numpy()

# Create a DataFrame with embeddings
embedding_df = pd.DataFrame(embeddings, index=all_entities.keys())

# Merge embeddings with labels
train_df['embedding'] = train_df.index.map(lambda x: embedding_df.loc[f'trans_{x}'].values)

In [1]:
#train_df.to_csv("train_df_new.csv")

In [2]:
torch.save(result.model.state_dict(), 'transe_model.pt')

NameError: name 'torch' is not defined

In [ ]:
#train_df['embedding']=pd.read_csv("train_df_new_W_EMBEDDINGS.csv")['embedding']
train_df['embedding']=pd.read_csv("/kaggle/input/train-df-jan13-embeddings/train_df_new_13Jan2024.csv")['embedding']

In [ ]:
!pip install python-louvain

from community import community_louvain

# Calculate the Louvain community structure
partition = community_louvain.best_partition(G)

# Add the community assignments to the dataframe
train_df['community'] = train_df.index.map(lambda x: partition.get(f'trans_{x}', -1))

# Print or further analyze the community assignments
print(train_df[['community']])

In [ ]:
train_df.info()

In [ ]:
train_df.dropna()

In [ ]:
for col in train_df.columns:
    if train_df[col].dtype == 'object' and col != 'embedding':  # Skip 'embedding' column
        train_df[col] = train_df[col].astype('category')

In [ ]:
train_df=train_df.dropna()

In [ ]:
categorical_features = ['gender', 'community', 'time_of_day_category', 'job']
for col in categorical_features:
    train_df[col] = train_df[col].astype('category')

In [ ]:
train_df.describe()

In [3]:
#train_df.to_csv("train_df_new.csv")

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

# Fit and transform the 'gender' column in both training and validation data
train_df['gender_encoded'] = le.fit_transform(train_df['gender'])


X =  train_df[['amt', 'distance', 'age', 'gender_encoded','community','time_of_day_category','job']]#'predicted_labels'
y = train_df['is_fraud']



In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix
import xgboost as xgb
import matplotlib.pyplot as plt
import pandas as pd

# One-hot encode categorical features
X_encoded = pd.get_dummies(X, columns=['time_of_day_category'])#, 'job'])
features = ['gender_encoded', 'community']
for col in features:
    X_encoded[col] = X_encoded[col].astype('int')

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Initialize the XGBoost model
model = xgb.XGBClassifier(random_state=42,enable_categorical=True)

# Perform cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='f1')

print(f"Cross-Validation F1 Scores: {scores}")
print(f"Mean F1 Score: {scores.mean()}")

# Train the model on the training data
model.fit(X_train, y_train)

# Predict on the validation set
y_val_pred = model.predict(X_val)

# Evaluate the model
print(f"Classification Report:\n{classification_report(y_val, y_val_pred)}")
print(f"Confusion Matrix:\n{confusion_matrix(y_val, y_val_pred)}")

# Plot feature importance
xgb.plot_importance(model)
plt.title('Feature Importance')
plt.show()


In [ ]:
from sklearn.metrics import precision_recall_fscore_support, precision_score, recall_score, f1_score

y_val_pred_proba = model.predict_proba(X_val)[:, 1]
cutoffs = np.linspace(0, 1, 101)
precision = []
recall = []
f1 = []

for cutoff in cutoffs:
    y_val_pred = (y_val_pred_proba >= cutoff).astype(int)
    precision.append(precision_score(y_val, y_val_pred))
    recall.append(recall_score(y_val, y_val_pred))
    f1.append(f1_score(y_val, y_val_pred))

plt.figure(figsize=(10, 6))
plt.plot(cutoffs, precision, label='Precision')
plt.plot(cutoffs, recall, label='Recall')
plt.plot(cutoffs, f1, label='F1 Score')
plt.xlabel('Cutoff Probability')
plt.ylabel('Score')
plt.title('Precision, Recall, and F1 Score vs. Cutoff Probability')
plt.legend()
plt.grid(True)
plt.show()


best_cutoff = cutoffs[np.argmax(f1)]
print(f"Best cutoff for F1 Score: {best_cutoff}")

y_val_pred = (y_val_pred_proba >= best_cutoff).astype(int)
print(classification_report(y_val,y_val_pred))


xgb.plot_importance(model)
plt.title('Feature Importance')
plt.show()

In [ ]:
test_df = pd.read_csv('/kaggle/input/fraud-detection/fraudTest.csv')

In [ ]:
test_df.columns

In [ ]:
import torch
from pykeen.models import TransE
from pykeen.triples import TriplesFactory

# Initialize the model with the same parameters used during training
model = TransE(
    triples_factory=train_tf,  # Add the triples_factory argument
    embedding_dim=50,  # Ensure this matches the embedding dimension used during training
)


# Load the saved state dictionary
model.load_state_dict(torch.load('transe_model.pt'))

# Move the model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Example: Get embeddings for a specific entity
entity_id = train_tf.entity_to_id['trans_0']  # Replace 'trans_0' with your entity ID
entity_embedding = model.entity_representations[0]

print(entity_embedding)


In [ ]:
TransE_model=model

In [ ]:
import networkx as nx
from community import community_louvain
test_df['trans_date_trans_time'] = pd.to_datetime(test_df['trans_date_trans_time'])
test_df['time_of_day'] = (test_df['trans_date_trans_time'].dt.hour * 60) + test_df['trans_date_trans_time'].dt.minute
test_df['time_of_day_category'] = pd.cut(test_df['time_of_day'], bins=[0, 6*60, 12*60, 18*60, 24*60], labels=['Night', 'Morning', 'Afternoon', 'Evening'], right=False)
test_df['unix_time_c'] = pd.to_datetime(test_df['unix_time'], unit='s')
test_df['time_of_day_uc'] = (test_df['unix_time_c'].dt.hour * 60) + test_df['unix_time_c'].dt.minute
test_df['time_of_day_category_uc'] = pd.cut(test_df['time_of_day_uc'], bins=[0, 6*60, 12*60, 18*60, 24*60], labels=['Night', 'Morning', 'Afternoon', 'Evening'], right=False)

test_df = test_df.drop(['Unnamed: 0', 'trans_date_trans_time','trans_num', 'unix_time'],axis=1)
test_df['distance']= (test_df['lat'] - test_df['merch_lat'])**2 + (test_df['long'] - test_df['merch_long'])**2
#test_df['merchant'] = test_df['merchant'].apply(clean_text)
#test_df['category'] = test_df['category'].apply(clean_text)
test_df['city_state'] = test_df['city'] + '_' + test_df['state']
test_df = test_df.dropna(subset=['dob'])
test_df['age'] = test_df['dob'].apply(calculate_age)

categorical_cols = ['category', 'gender', 'job','city','state','merchant','street','city_state']
for col in categorical_cols:
    test_df[col] = test_df[col].astype('category')

test_df.drop(['merch_lat', 'merch_long','lat', 'long','street','zip','city_pop','dob'],axis=1,inplace=True)


# Create graph for test data
G_test = nx.Graph()
for index, row in test_df.iterrows():
    trans_id = f'trans_{index}'
    G_test.add_node(trans_id, label='transaction')
    G_test.add_node(row['merchant'], label='merchant')
    G_test.add_node(row['category'], label='category')
    G_test.add_node(row['city'], label='city')
    G_test.add_node(row['state'], label='state')
    G_test.add_edge(trans_id, row['merchant'], relationship='made_at')
    G_test.add_edge(trans_id, row['category'], relationship='belongs_to')
    G_test.add_edge(trans_id, row['city'], relationship='located_in')
    G_test.add_edge(trans_id, row['state'], relationship='in')
    G_test.add_node(row['time_of_day_category'], label='time')
    G_test.add_edge(trans_id, row['time_of_day_category'], relationship='time')
    #G_test.add_node(row['cc_num'], label='cc_num')
    #G_test.add_edge(trans_id, row['cc_num'], relationship='card')
    G_test.add_node(row['job'], label='job')
    G_test.add_edge(trans_id, row['job'], relationship='job')
# Community detection for test data
partition_test = community_louvain.best_partition(G_test)
test_df['community'] = test_df.index.map(lambda x: partition_test.get(f'trans_{x}', -1))

# Label Propagation for test data (using the trained model)
transaction_nodes_test = [node for node in G_test.nodes() if str(node).startswith('trans_')]
#graph_laplacian_test = nx.laplacian_matrix(G_test, nodelist=transaction_nodes_test).toarray()
#predicted_labels_test = label_prop_model.predict(graph_laplacian_test) # Use the trained model here
#test_df['predicted_labels'] = predicted_labels_test


In [ ]:
print(dir(model))


In [ ]:
from pykeen.triples.utils import get_entities
entity_to_id = result.training.entity_to_id
relation_to_id = result.training.relation_to_id


# Create triples for the test data
test_triples = []
for index, row in test_df.iterrows():
    trans_id = f'trans_{index}'
    test_triples.append((trans_id, 'made_at', row['merchant']))
    test_triples.append((trans_id, 'belongs_to', row['category']))
    test_triples.append((trans_id, 'located_in', row['city']))
    test_triples.append((trans_id, 'in', row['state']))
    test_triples.append((trans_id, 'in', row['time_of_day_category']))
    test_triples.append((trans_id, 'job', row['job']))
    #test_triples.append((trans_id, 'card', row['cc_num']))
test_triples = np.array(test_triples)

# Remove malformed entries (if any)
valid_test_triples = [triple for triple in test_triples if len(triple) == 3 and all(isinstance(item, str) for item in triple)]
test_triples = np.array(valid_test_triples)


# Create a TriplesFactory for the test data, handling unknown entities
test_tf = TriplesFactory.from_labeled_triples(
    triples=test_triples,
    entity_to_id=entity_to_id,  # Use the entity to ID mapping from training
    relation_to_id=result.training.relation_to_id,  # Use relation mapping from training
    create_inverse_triples=False, # Assuming you don't need inverse triples
)



In [ ]:
from pykeen.triples.utils import get_entities
import numpy as np

# Access the entity and relation mappings from train_tf
entity_to_id = train_tf.entity_to_id
relation_to_id = train_tf.relation_to_id

# Create triples for the test data
test_triples = []
for index, row in test_df.iterrows():
    trans_id = f'trans_{index}'
    test_triples.append((trans_id, 'made_at', row['merchant']))
    test_triples.append((trans_id, 'belongs_to', row['category']))
    test_triples.append((trans_id, 'located_in', row['city']))
    test_triples.append((trans_id, 'in', row['state']))
    test_triples.append((trans_id, 'in', row['time_of_day_category']))
    test_triples.append((trans_id, 'job', row['job']))
    #test_triples.append((trans_id, 'card', row['cc_num']))
test_triples = np.array(test_triples)

# Remove malformed entries (if any)
valid_test_triples = [triple for triple in test_triples if len(triple) == 3 and all(isinstance(item, str) for item in triple)]
test_triples = np.array(valid_test_triples)

# Create a TriplesFactory for the test data, handling unknown entities
test_tf = TriplesFactory.from_labeled_triples(
    triples=test_triples,
    entity_to_id=entity_to_id,  # Use the entity to ID mapping from training
    relation_to_id=relation_to_id,  # Use relation mapping from training
    create_inverse_triples=False, # Assuming you don't need inverse triples
)


In [ ]:

# Get entity embeddings for the test data
# Get all entity IDs present in the test data
all_test_entity_ids = set(test_tf.entities_to_ids(get_entities(test_tf.mapped_triples))) # Instead of test_tf.get_entities()

# Filter entity embeddings for entities present in test data
test_entity_indices = torch.tensor([test_tf.entity_to_id[entity] for entity in test_tf.entity_to_id if test_tf.entity_to_id[entity] in all_test_entity_ids], device=result.model.device)
test_embeddings = result.model.entity_representations[0](indices=test_entity_indices).cpu().detach().numpy()

# Create a DataFrame with embeddings for the test data, using filtered entity IDs
test_embedding_df = pd.DataFrame(test_embeddings, index=test_entity_indices.cpu().numpy())

# Merge embeddings with the test DataFrame, using numeric entity IDs
test_df['embedding'] = test_df.index.map(lambda x: test_embedding_df.loc[test_tf.entity_to_id.get(f'trans_{x}')].values if test_tf.entity_to_id.get(f'trans_{x}') is not None else None)

In [ ]:
from pykeen.triples.utils import get_entities
import numpy as np

# Access the entity and relation mappings from train_tf
entity_to_id = train_tf.entity_to_id
relation_to_id = train_tf.relation_to_id

# Create triples for the test data
test_triples = []
for index, row in test_df.iterrows():
    trans_id = f'trans_{index}'
    test_triples.append((trans_id, 'made_at', row['merchant']))
    test_triples.append((trans_id, 'belongs_to', row['category']))
    test_triples.append((trans_id, 'located_in', row['city']))
    test_triples.append((trans_id, 'in', row['state']))
    test_triples.append((trans_id, 'in', row['time_of_day_category']))
    test_triples.append((trans_id, 'job', row['job']))
    #test_triples.append((trans_id, 'card', row['cc_num']))
test_triples = np.array(test_triples)

# Remove malformed entries (if any)
valid_test_triples = [triple for triple in test_triples if len(triple) == 3 and all(isinstance(item, str) for item in triple)]
test_triples = np.array(valid_test_triples)

# Create a TriplesFactory for the test data, handling unknown entities
test_tf = TriplesFactory.from_labeled_triples(
    triples=test_triples,
    entity_to_id=entity_to_id,  # Use the entity to ID mapping from training
    relation_to_id=relation_to_id,  # Use relation mapping from training
    create_inverse_triples=False, # Assuming you don't need inverse triples
)

# Filter entity embeddings for entities present in test data
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
test_entity_indices = torch.tensor(
    [test_tf.entity_to_id[entity] for entity in test_tf.entity_to_id if test_tf.entity_to_id[entity] in all_test_entity_ids],
    device=device
)
test_embeddings = model.entity_representations[0]

print(test_embeddings)


In [ ]:
print("Shape of test_embeddings:", test_embeddings.shape)
print("Shape of test_entity_indices:", test_entity_indices.shape)

# Create a DataFrame with embeddings for the test data, using filtered entity IDs
test_embedding_df = pd.DataFrame(test_embeddings, index=test_entity_indices.cpu().numpy())

# Merge embeddings with the test DataFrame, using numeric entity IDs
test_df['embedding'] = test_df.index.map(lambda x: test_embedding_df.loc[test_tf.entity_to_id.get(f'trans_{x}')].values if test_tf.entity_to_id.get(f'trans_{x}') is not None else None)

print(test_df)

In [ ]:
test_df['gender_encoded'] = le.transform(test_df['gender'])

In [ ]:
test_df.to_csv("test_df_new.csv")

In [ ]:
X_train.columns

In [ ]:
test_df_trim=test_df[['amt', 'distance', 'age', 'gender_encoded', 'community', 'time_of_day_category', 'job']]
test_df_encoded = pd.get_dummies(test_df_trim, columns=['time_of_day_category', 'job'])
for col in features:
    test_df_encoded[col] = test_df_encoded[col].astype('int')

missing_cols = set(X_train.columns) - set(test_df_encoded.columns)
for col in missing_cols:
    test_df_encoded[col] = 0

test_df_encoded = test_df_encoded[X_train.columns]


X_test = test_df_encoded
y_test_pred_proba = model.predict_proba(X_test)[:, 1]
y_test_pred = (y_test_pred_proba >= best_cutoff).astype(int) # Use the best cutoff from training

# Evaluate the model on the test data
print(classification_report(test_df['is_fraud'], y_test_pred)) # Assuming 'is_fraud' column exists in test_df
print(confusion_matrix(test_df['is_fraud'], y_test_pred))

##### Use stacking

In [ ]:
X =  train_df[['amt', 'distance', 'age', 'gender_encoded','community','time_of_day_category','job']]#'predicted_labels'
y = train_df['is_fraud']

# One-hot encode categorical features
X_encoded = pd.get_dummies(X, columns=['time_of_day_category', 'job'])
features = ['gender_encoded', 'community']
for col in features:
    X_encoded[col] = X_encoded[col].astype('int')

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X_encoded, y, test_size=0.2, random_state=42)


In [ ]:
#!pip install scikit-learn==1.2.2
#!pip install xgboost==1.7.5
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression



base_learners = [
    ('rf',RandomForestClassifier(n_estimators=100)),
    ('xgb', xgb.XGBClassifier(random_state=42, enable_categorical=True))
]

meta_learner = LogisticRegression()
stacking_model = StackingClassifier(estimators=base_learners, final_estimator=meta_learner)

stacking_model.fit(X_train, y_train)


In [ ]:
#model = xgb.XGBClassifier(early_stopping_rounds=10, eval_metric='auc',enable_categorical=True)
#model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=True)

# Make predictions on the validation set
y_val_pred = stacking_model.predict(X_val)

print(f"classification report \n {classification_report(y_val, y_val_pred)}")
print(f"confusion_matrix \n {confusion_matrix(y_val, y_val_pred)}")


In [ ]:
for col in X_train.columns:
    if col not in X_test.columns:
        X_test[col] = 0

In [ ]:
X_test = X_test[X_train.columns]


In [ ]:
y_pred = stacking_model.predict(X_test)
y_test=test_df['is_fraud']
print(f"classification report \n {classification_report(y_test, y_pred)}")
print(f"confusion_matrix \n {confusion_matrix(y_test, y_pred)}")

In [ ]:
# Install LIME
#!pip install lime

import lime
import lime.lime_tabular
import numpy as np

# Assuming you have a trained stacking classifier named 'stacking_clf'
# and your training data is in 'X_train' and 'y_train'

# Create a LIME explainer
explainer = lime.lime_tabular.LimeTabularExplainer(
    training_data=np.array(X_train),
    feature_names=X_train.columns,
    class_names=['Not Fraud', 'Fraud'],
    mode='classification'
)

# Select an instance to explain
instance_idx = 2000
instance = X_train.iloc[instance_idx]

# Generate explanation
exp = explainer.explain_instance(
    data_row=instance,
    predict_fn=stacking_model.predict_proba
)

# Display explanation
exp.show_in_notebook(show_table=True, show_all=False)

With upsampling of minority class

In [ ]:

df_majority = train_df[train_df.is_fraud==0]
df_minority = train_df[train_df.is_fraud==1]

# Upsample minority class
from sklearn.utils import resample
df_minority_upsampled = resample(df_minority,
                                 replace=True,     # sample with replacement
                                 n_samples=int(0.05*len(train_df)),    # to match majority class
                                 random_state=123) # reproducible results

# Combine majority class with upsampled minority class
train_df_upsampled = pd.concat([df_majority, df_minority_upsampled])

# Display new class counts
train_df_upsampled.is_fraud.value_counts()



In [ ]:

le = LabelEncoder()
train_df_upsampled['gender_encoded'] = le.fit_transform(train_df_upsampled['gender'])

X =  train_df_upsampled[['amt', 'distance', 'age', 'gender_encoded','community','time_of_day_category','job']]
y = train_df_upsampled['is_fraud']
X_encoded = pd.get_dummies(X, columns=['time_of_day_category', 'job'])
X_train, X_val, y_train, y_val = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Initialize and train the XGBoost model
model = xgb.XGBClassifier(random_state=42, enable_categorical=True)
model.fit(X_train, y_train)
y_val_pred = model.predict(X_val)

print(f"classification report \n {classification_report(y_val, y_val_pred)}")
print(f"confusion_matrix \n {confusion_matrix(y_val, y_val_pred)}")

xgb.plot_importance(model)
plt.title('Feature Importance')
plt.show()

y_val_pred_proba = model.predict_proba(X_val)[:, 1]
cutoffs = np.linspace(0, 1, 101)
precision = []
recall = []
f1 = []

for cutoff in cutoffs:
    y_val_pred = (y_val_pred_proba >= cutoff).astype(int)
    precision.append(precision_score(y_val, y_val_pred))
    recall.append(recall_score(y_val, y_val_pred))
    f1.append(f1_score(y_val, y_val_pred))

plt.figure(figsize=(10, 6))
plt.plot(cutoffs, precision, label='Precision')
plt.plot(cutoffs, recall, label='Recall')
plt.plot(cutoffs, f1, label='F1 Score')
plt.xlabel('Cutoff Probability')
plt.ylabel('Score')
plt.title('Precision, Recall, and F1 Score vs. Cutoff Probability')
plt.legend()
plt.grid(True)
plt.show()

best_cutoff = cutoffs[np.argmax(f1)]
print(f"Best cutoff for F1 Score: {best_cutoff}")

y_val_pred = (y_val_pred_proba >= best_cutoff).astype(int)
print(classification_report(y_val,y_val_pred))

xgb.plot_importance(model)
plt.title('Feature Importance')
plt.show()

print("Testing\n")
test_df_trim=test_df[['amt', 'distance', 'age', 'gender_encoded', 'community', 'time_of_day_category', 'job']]
test_df_encoded = pd.get_dummies(test_df_trim, columns=['time_of_day_category', 'job'])

missing_cols = set(X_train.columns) - set(test_df_encoded.columns)
for col in missing_cols:
    test_df_encoded[col] = 0

test_df_encoded = test_df_encoded[X_train.columns]


X_test = test_df_encoded
y_test_pred_proba = model.predict_proba(X_test)[:, 1]
y_test_pred = (y_test_pred_proba >= best_cutoff).astype(int) # Use the best cutoff from training

# Evaluate the model on the test data
print(classification_report(test_df['is_fraud'], y_test_pred)) # Assuming 'is_fraud' column exists in test_df
print(confusion_matrix(test_df['is_fraud'], y_test_pred))

In [ ]:
import torch
from pykeen.models import TransE
from pykeen.triples import TriplesFactory

# Initialize the model with the same parameters used during training
model = TransE(
    triples_factory=train_tf,  # Add the triples_factory argument
    embedding_dim=50,  # Ensure this matches the embedding dimension used during training
)


# Load the saved state dictionary
model.load_state_dict(torch.load('transe_model.pt'))

# Move the model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Example: Get embeddings for a specific entity
entity_id = train_tf.entity_to_id['trans_0']  # Replace 'trans_0' with your entity ID
entity_embedding = model.entity_representations[0]

print(entity_embedding)


In [ ]:
entity_to_id = model.training.entity_to_id

# Create triples for the test data
test_triples = []
for index, row in test_df.iterrows():
    trans_id = f'trans_{index}'
    test_triples.append((trans_id, 'made_at', row['merchant']))
    test_triples.append((trans_id, 'belongs_to', row['category']))
    test_triples.append((trans_id, 'located_in', row['city']))
    test_triples.append((trans_id, 'in', row['state']))
    test_triples.append((trans_id, 'in', row['time_of_day_category']))
    test_triples.append((trans_id, 'job', row['job']))
    #test_triples.append((trans_id, 'card', row['cc_num']))
test_triples = np.array(test_triples)

# Remove malformed entries (if any)
valid_test_triples = [triple for triple in test_triples if len(triple) == 3 and all(isinstance(item, str) for item in triple)]
test_triples = np.array(valid_test_triples)

# Create a TriplesFactory for the test data, handling unknown entities
test_tf = TriplesFactory.from_labeled_triples(
    triples=test_triples,
    entity_to_id=entity_to_id,  # Use the entity to ID mapping from training
    relation_to_id=result.training.relation_to_id,  # Use relation mapping from training
    create_inverse_triples=False,  # Assuming you don't need inverse triples
)